In [1]:
import pandas as pd
import re
import operator
import string
import nltk 
from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree
from nltk.corpus import stopwords

In [31]:
ALLAWARDS = [['best', 'picture', 'drama'], ['actor', 'actress'],
['best', 'picture', 'musical', 'comedy'], ['actor', 'actress'],
['best', 'picture', 'drama', 'actress'], ['actor'],
['best', 'picture', 'drama', 'actor'], ['actress'],
['best', 'picture', 'musical', 'comedy', 'actress'], ['actor'],
['best', 'picture', 'musical', 'comedy', 'actor'], ['actress'],
['best', 'picture', 'supporting', 'actress'], ['actor'],
['best', 'picture', 'supporting', 'actor'], ['actress'],
['best', 'picture', 'director'], [],
['best', 'picture', 'screenplay'], [],
['best', 'picture', 'animated'], [],
['best', 'foreign', 'language'], [],
['best', 'picture', 'score'], [],
['best', 'picture', 'song'],[],
['best', 'drama', 'series'], [],
['best', 'musical', 'comedy', 'series'], [],
['best', 'television', 'picture'], ['actor', 'actress'],
['best', 'tv', 'picture', 'actress'], ['actor'], #discrepenacy might break in 2015
['best', 'television', 'picture', 'actor'], ['actress'] ,
['best', 'television', 'drama', 'series', 'actress'], ['actor'],
['best', 'tv', 'drama', 'actor'], ['actress'], #might break in 2015
['best', 'musical', 'comedy', 'series', 'tv', 'actress'], ['actor'],
['best', 'musical', 'comedy', 'series', 'tv', 'actor'], ['actress'],
['best', 'tv', 'supporting', 'actress'], ['actor'],
['best', 'tv', 'supporting', 'actor'], ['actress'],
['cecil', 'award'], []] 

In [45]:
categories = [ALLAWARDS[i] for i in range(0, len(ALLAWARDS), 2)]
categories

[['best', 'picture', 'drama'],
 ['best', 'picture', 'musical', 'comedy'],
 ['best', 'picture', 'drama', 'actress'],
 ['best', 'picture', 'drama', 'actor'],
 ['best', 'picture', 'musical', 'comedy', 'actress'],
 ['best', 'picture', 'musical', 'comedy', 'actor'],
 ['best', 'picture', 'supporting', 'actress'],
 ['best', 'picture', 'supporting', 'actor'],
 ['best', 'picture', 'director'],
 ['best', 'picture', 'screenplay'],
 ['best', 'picture', 'animated'],
 ['best', 'foreign', 'language'],
 ['best', 'picture', 'score'],
 ['best', 'picture', 'song'],
 ['best', 'drama', 'series'],
 ['best', 'musical', 'comedy', 'series'],
 ['best', 'television', 'picture'],
 ['best', 'tv', 'picture', 'actress'],
 ['best', 'television', 'picture', 'actor'],
 ['best', 'television', 'drama', 'series', 'actress'],
 ['best', 'tv', 'drama', 'actor'],
 ['best', 'musical', 'comedy', 'series', 'tv', 'actress'],
 ['best', 'musical', 'comedy', 'series', 'tv', 'actor'],
 ['best', 'tv', 'supporting', 'actress'],
 ['best

In [2]:
gg = pd.read_json('gg2013.json', orient='records')
gg1 = pd.read_json('gg2015.json', orient='records')

In [3]:
copy = gg.copy(deep=True)

In [43]:
copy.head()

,text,user,id,timestamp_ms
0,JLos dress,"{'screen_name': 'Dozaaa_xo', 'id': 557374298}",290620657987887104,2013-01-14 00:45:38
1,Whats making Sofia Vergaras boobs stay like th...,"{'screen_name': 'theAmberShow', 'id': 14648726}",290620657887219713,2013-01-14 00:45:38
2,Kerry Washington is EVERYTHING Dying over her ...,"{'screen_name': 'SweetyPW', 'id': 35498686}",290620657828524032,2013-01-14 00:45:38
3,Anne Hathaway has got me living,"{'screen_name': '_NicoleEdwards', 'id': 144430...",290620657799159809,2013-01-14 00:45:38
4,Jennifer Lopezs lace dress Thoughts,"{'screen_name': 'lolaogunnaike', 'id': 134953223}",290620657778188288,2013-01-14 00:45:38


In [4]:
def removePunc(x):
    x = re.sub(r'[@#]\w+', '', x) #taking out hashtags and @ 
    x = re.sub(r'(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)', '', x) #taking out links 
    x = re.sub(r'[!?\.,\'\":()]+', '', x) #taking out punctuation i.e ? ! . ' and " 
    x = re.sub(r'(RT|rt) ', '', x) #taking out the initial "RT "
    x= re.sub(r'(g|G)olden (g|G)lobes*', '', x)
    return x.strip()
# copy['text'] = copy['text'].str.lower()
copy['text'] = copy['text'].apply(lambda x: removePunc(str(x)))

In [60]:
#takes out duplicate retweets 
test = list(copy['text'].values)
print(len(test))
s = list(set(test))
print(len(s))
df = pd.DataFrame(s, columns=['text'])

174643
118369


In [61]:
def findRows(x, contains, ncontains):
    s = "^"
    for word in contains:
        s += f'(?=.*{word})'
    for word in ncontains:
        s += f'(?!.*{word})'
    s += '.*$'
    if bool(re.match(s, x)):
        return True
    else:
        False
con = ['best', 'tv', 'supporting', 'actress']
noncon = []
df['Rows'] = df['text'].apply(lambda x: findRows(x, con, noncon))
print(len(df))
ccc = df[~df['Rows'].isnull()]
print(len(ccc))

118369
2


In [69]:
p = 0
for category in categories:
    con = category
    noncon = []
    df['Rows'] = df['text'].apply(lambda x: findRows(x, con, noncon))
    print(len(df))
    ccc = df[~df['Rows'].isnull()]
    print('subset of tweets filtered by category length: ', category , len(ccc))
    dum = ccc.loc[ccc['text'].str.contains('presents') |
              ccc['text'].str.contains('presenting') | 
              ccc['text'].str.contains('presented') |
             ccc['text'].str.contains('introducing') |
             ccc['text'].str.contains('introduces') |
             ccc['text'].str.contains('introduced') |  
             ccc['text'].str.contains('awarding') | 
             ccc['text'].str.contains('gives') | 
             ccc['text'].str.contains('giving') |
            ccc['text'].str.contains('gave')
                  
            ]
    print('tweets with actual magic words: ', len(dum))
    if(len(dum) > 0):
        dum.to_csv(f'presenters{p}.csv', index=False)
        p += 1

118369
subset of tweets filtered by category length:  ['best', 'picture', 'drama'] 51
tweets with actual magic words:  3
118369
subset of tweets filtered by category length:  ['best', 'picture', 'musical', 'comedy'] 37
tweets with actual magic words:  0
118369
subset of tweets filtered by category length:  ['best', 'picture', 'drama', 'actress'] 2
tweets with actual magic words:  0
118369
subset of tweets filtered by category length:  ['best', 'picture', 'drama', 'actor'] 4
tweets with actual magic words:  0
118369
subset of tweets filtered by category length:  ['best', 'picture', 'musical', 'comedy', 'actress'] 3
tweets with actual magic words:  0
118369
subset of tweets filtered by category length:  ['best', 'picture', 'musical', 'comedy', 'actor'] 6
tweets with actual magic words:  0
118369
subset of tweets filtered by category length:  ['best', 'picture', 'supporting', 'actress'] 5
tweets with actual magic words:  0
118369
subset of tweets filtered by category length:  ['best', 'pi

In [62]:
dum = ccc.loc[ccc['text'].str.contains('presents') |
              ccc['text'].str.contains('presenting') | 
             ccc['text'].str.contains('introducing') |
             ccc['text'].str.contains('introduces') |
             ccc['text'].str.contains('awarding') | 
             ccc['text'].str.contains('gives') | 
             ccc['text'].str.contains('giving')]
len(dum)

1

In [63]:
dum.to_csv('presents.csv', index=False)

In [27]:
text = 'Lancome Ambassadress Julia Roberts looks stunning as she presents the best motion picture drama at the'

In [30]:
nltk_results = ne_chunk(pos_tag(word_tokenize(text)))
for nltk_result in nltk_results:
    print(nltk_result)
    if type(nltk_result) == Tree:
        name = ''
        for nltk_result_leaf in nltk_result.leaves():
            name += nltk_result_leaf[0] + ' '
        print ('Type: ', nltk_result.label(), 'Name: ', name)

(PERSON Lancome/NNP)
Type:  PERSON Name:  Lancome 
('Ambassadress', 'NNP')
('Julia', 'NNP')
('Roberts', 'NNP')
('looks', 'VBZ')
('stunning', 'VBG')
('as', 'IN')
('she', 'PRP')
('presents', 'VBZ')
('the', 'DT')
('best', 'JJS')
('motion', 'NN')
('picture', 'NN')
('drama', 'NN')
('at', 'IN')
('the', 'DT')
